# EXERCISE 6
Number 3 – VGG16 Architecture with Cat and Dog Dataset


In [1]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras import regularizers
import numpy as np

In [ ]:
import zipfile

# Define the paths for the zip files and extraction directories
training_zip_path = '/content/training_set.zip'
training_extract_dir = '/content/training_set'
test_zip_path = '/content/test_set.zip'
test_extract_dir = '/content/test_set'

# Create extraction directories if they don't exist
os.makedirs(training_extract_dir, exist_ok=True)
os.makedirs(test_extract_dir, exist_ok=True)

# Extract training_set.zip
print(f"Extracting {training_zip_path} to {training_extract_dir}...")
with zipfile.ZipFile(training_zip_path, 'r') as zip_ref:
    zip_ref.extractall(training_extract_dir)
print("Training set extracted.")

# Extract test_set.zip
print(f"Extracting {test_zip_path} to {test_extract_dir}...")
with zipfile.ZipFile(test_zip_path, 'r') as zip_ref:
    zip_ref.extractall(test_extract_dir)
print("Test set extracted.")

In [8]:
# DATASET DIRECTORY CONFIGURATION
train_dir = "/content/training_set/training_set"
test_dir = "/content/test_set/test_set"

In [9]:
# IMAGE PARAMETERS
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [22]:
# DATA PREPROCESSING & AUGMENTATION
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 6404 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [23]:
# SIMPLE CNN MODEL ARCHITECTURE

# Some modifications are applied
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers to reduce overfitting
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
# Configure the model optimizers, loss function, and metrics
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [25]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 256s 1s/step - accuracy: 0.5037 - loss: 0.7532 - val_accuracy: 0.4997 - val_loss: 0.6930
Epoch 2/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 274s 1s/step - accuracy: 0.5225 - loss: 0.6917 - val_accuracy: 0.5734 - val_loss: 0.6694
Epoch 3/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 248s 1s/step - accuracy: 0.5871 - loss: 0.6719 - val_accuracy: 0.6252 - val_loss: 0.6623
Epoch 4/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 251s 1s/step - accuracy: 0.6001 - loss: 0.6704 - val_accuracy: 0.6533 - val_loss: 0.6102
Epoch 5/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 247s 1s/step - accuracy: 0.6490 - loss: 0.6224 - val_accuracy: 0.6771 - val_loss: 0.6040
Epoch 6/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 248s 1s/step - accuracy: 0.6802 - loss: 0.6024 - val_accuracy: 0.6814 - val_loss: 0.5924
Epoch 7/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 248s 1s/step - accuracy: 0.6812 - loss: 0.5918 - val_accuracy: 0.7239 - val_loss: 0.5424
Epoch 8/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 250s 1s/step - accuracy: 0.7110 - loss: 0.5631 - val_accu

In [26]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 19s 300ms/step - accuracy: 0.8065 - loss: 0.4347
Test Accuracy: 0.7711319923400879


In [27]:
# SAVE THE MODEL
model.save('exercise_6_custom_Esponilla-Valdez.h5')

In [40]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(128, 128))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # shape: (1,128,128,3)
    pred = model.predict(img_array)[0,0]
    label = "Cat" if pred >= 0.5 else "Dog"
    confidence = pred if pred >= 0.5 else 1 - pred
    print(f"Prediction: {label} (confidence: {confidence:.2f})")

In [41]:
# Example usage:
predict_image("/content/test_set/test_set/cats/cat.4001.jpg")
predict_image("/content/test_set/test_set/dogs/dog.4001.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Prediction: Cat (confidence: 0.84)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Prediction: Cat (confidence: 0.72)
